# 参数管理
- 访问参数，用于调试、诊断和可视化
- 参数初始化
- 在不同模型组件间共享参数

In [2]:
import torch

from torch import nn

In [3]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0046],
        [-0.0713]], grad_fn=<AddmmBackward0>)

## 参数访问
- 当通过`Sequential`类定义模型时，可以**通过索引**来访问模型的任意层
- 参数是**复合的对象**，包含**值、梯度和额外信息**

In [4]:
print(net[2].state_dict())
print()

print(type(net[2].bias))
print()
print(net[2].bias)
print()
print(net[2].bias.data)

OrderedDict([('weight', tensor([[-0.2660, -0.1183, -0.1681, -0.2617,  0.2256,  0.1269,  0.0813,  0.3273]])), ('bias', tensor([-0.1856]))])

<class 'torch.nn.parameter.Parameter'>

Parameter containing:
tensor([-0.1856], requires_grad=True)

tensor([-0.1856])


In [5]:
net[2].weight.grad == None  # 还没有做反向计算

True

## 一次性访问所有参数
- `named_parameters()`：在模型的每一层中查找参数并返回一个**迭代器**，**生成`(name, parameter)`的二元组**

In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print()
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'].data

tensor([-0.1856])

## 从嵌套块收集参数
- `.add_module(name, module)`是pytorch中`nn.Module`的方法，用于在现有网络中动态添加新的子模块

In [11]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3022],
        [-0.3021]], grad_fn=<AddmmBackward0>)

- 设计了网络后，看**如何工作的**

In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


- 层是分层嵌套的，故也可以通过**嵌套列表索引一样**访问

In [23]:
rgnet[0][1][0].bias.data

tensor([ 0.1353, -0.4055, -0.1863, -0.4820, -0.4047,  0.1192,  0.2199,  0.3586])

## 参数初始化

### 内置初始化
- `net.apply`：**递归地遍历并应用**自定义初始化或其他操作到**模型中的所有模块**的方法
    - 可以在复杂的神经网络中，快速**对特定类型的层**进行操作
- `net[0].weight.data[0]`取的是第一行的权重，包含该层第一个**输出节点**对所有输入节点的连接权重
- `net[0].bias.data[0]`取的是**第一个输出节点**的偏置

In [13]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0069, -0.0144, -0.0026,  0.0213]), tensor(0.))

- 从**API**角度来看，可以如下这么做——**但是！**不能这样做

In [16]:
def init_constant_forbid(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant_forbid)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

- 对某些块应用**不同**的初始化

In [17]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([ 0.3449,  0.6463,  0.4717, -0.5628])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


### 自定义初始化
- 例子知识为了展示
    - `m.named_parameters()][0]`中的`[0]`保证只获取**第一对**`(name, param_shape)`，线性层通常只包含`weight`和`bias`参数，`[0]`确保只输出第一个
    - `nn.init.uniform`表示均匀分布
    - 因为`*`解包的作用，`print`会**逐个打印出元组内的每个元素**
    

In [24]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            'Init',
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5  # 这个写法！

net.apply(my_init)
net[0].weight.data[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([8, 8])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000,  0.0000, -0.0000],
        [-9.9931,  0.0000, -0.0000, -7.9628]])

更直接的方法——**直接设置**

In [20]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  0.5923,  0.9251,  0.9398])

## 参数绑定（共享）
有时希望在多个层间**共享参数**：可以定义一个稠密层，然后使用其参数来设置另一层的参数
- 第3和第5个神经网络层的参数是绑定的——不仅值相等，且**由相同的张量表示**（改名一个另一个也会**跟着改变**）
- 在反向传播阶段，**梯度会互相影响**
    - **梯度累加**：每层对损失的贡献都会计算一个梯度，因为共享相同的参数张量，给子计算的梯度会***累加*在同一张量上**
    - **更新相同的参数**：统一更新共享参数

In [25]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == shared.weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习
1. 使用5.1节中定义的`NestMLP`模型，访问各个层的参数。

In [29]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

# print('访问各个层的参数')
# net = NestMLP()
# for name, param in net.named_parameters():
#     print(f'parameter name:{name}, shape:{param.shape}, parameter:{param}')

print('访问net层的参数')
for name, param in net.net.named_parameters():
    print(f'parameter name:{name}, shape:{param.shape}, parameter:{param}')

print('访问linear层的参数')
for name, param in net.linear.named_parameters():
    print(f'parameter name:{name}, shape:{param.shape}, parameter:{param}')


访问net层的参数
parameter name:0.weight, shape:torch.Size([64, 20]), parameter:Parameter containing:
tensor([[-0.0511,  0.0768,  0.0607,  ...,  0.1370, -0.0483, -0.1949],
        [ 0.0039, -0.0216, -0.1054,  ...,  0.1504,  0.0050, -0.2224],
        [ 0.1375, -0.1107, -0.2031,  ...,  0.1635,  0.0671, -0.1086],
        ...,
        [-0.0564,  0.0200,  0.1294,  ...,  0.0462,  0.0958,  0.1335],
        [-0.0062,  0.0008, -0.1809,  ..., -0.2223,  0.1046,  0.1954],
        [ 0.0195, -0.1499, -0.2206,  ..., -0.1502,  0.0082, -0.2133]],
       requires_grad=True)
parameter name:0.bias, shape:torch.Size([64]), parameter:Parameter containing:
tensor([-0.0896,  0.1172,  0.0579, -0.2124,  0.2234, -0.1427,  0.1706, -0.1779,
        -0.0863, -0.0672,  0.1784, -0.1981, -0.1585, -0.2035, -0.2213,  0.0548,
        -0.1894, -0.0575,  0.2096, -0.0115,  0.1678,  0.0135,  0.0088, -0.1482,
         0.1532, -0.2022, -0.0273,  0.0917,  0.0760, -0.1379, -0.1597, -0.1613,
         0.1149, -0.1224,  0.2055, -0.0873,  

2. 查看初始化模块文档以了解不同的初始化方法。
- `torch.nn.init.uniform_(tensor, a=, b=)`
- `torch.nn.init.normal_(tensor, mean=, std=)`
- `torch.nn.init.constant_(tensor, val)`
- `torch.nn.init.ones_(tensor)`
- `torch.nn.init.zeros_(tensor)`
- `torch.nn.init.xavier_uniform_(tensor, gain=)`
- `torch.nn.init.xavier_normal(tensor, gain=)`
- ...

3. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
- `shared_layer`层的参数和梯度都是**相同**的，因为共享同一个参数
    - 损失函数`loss`也可以用`nn.functional.mse_loss`

In [35]:
input_size, hidden_size, output_size = 4, 8, 4
num_epochs, lr = 2, 0.01

shared_layer = nn.Linear(hidden_size, hidden_size)
MLP = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(),
                    shared_layer, nn.ReLU(),
                    shared_layer, nn.ReLU(),
                    nn.Linear(hidden_size, output_size)
                   )

X = torch.randn(1, input_size)
y = torch.randn(1, output_size)

trainer = torch.optim.SGD(MLP.parameters(), lr=lr)
loss = nn.MSELoss()

for epoch in range(num_epochs):
    trainer.zero_grad()
    l = loss(MLP(X), y)
    l.backward()
    trainer.step()
    for name, param in MLP.named_parameters():
        print(name,'\n', param.data, '\n', param.grad)
        print()
    print('epoch:{}, loss:{}'.format(epoch + 1, l.item()))
    print()

0.weight 
 tensor([[-0.2615, -0.3793, -0.0525, -0.4902],
        [-0.0181, -0.3714, -0.4674, -0.4863],
        [ 0.2014,  0.1035,  0.2618, -0.2640],
        [-0.3097,  0.2032, -0.1983,  0.0953],
        [-0.3038,  0.4684, -0.1896,  0.3220],
        [-0.2873, -0.4536, -0.0364, -0.3834],
        [ 0.3330, -0.0890,  0.1724, -0.4184],
        [-0.1871,  0.0408,  0.0925, -0.2612]]) 
 tensor([[-0.0350, -0.0252, -0.0342, -0.0042],
        [ 0.0243,  0.0175,  0.0238,  0.0029],
        [-0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0063, -0.0046, -0.0062, -0.0008],
        [-0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.0132,  0.0095,  0.0129,  0.0016],
        [-0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000]])

0.bias 
 tensor([ 0.0979, -0.4704, -0.2744,  0.0315, -0.1829,  0.3130, -0.4065, -0.2642]) 
 tensor([ 0.0226, -0.0157,  0.0000,  0.0041,  0.0000, -0.0085,  0.0000,  0.0000])

2.weight 
 tensor([[ 0.2578, -0.2951, -0.3418,  0.2458,  0.0512,  0.2972, 

4. 为什么共享参数是个好主意？
- **节约内存**：共享参数可以减少模型中**需要存储的参数数量**，从而减少内存占用
- **加速收敛**：共享参数可以让**模型更加稳定**，加速收敛
- **提高泛化能力**：共享参数可以帮助模型更好**捕捉数据中的共性**，提高模型的泛化能力
- **加强模型的可解释性**：共享参数可以让模型**更加简洁明了**，加强模型的可解释性

# 延后初始化 
defers initialization：直到数据第一次通过模型传递时，框架才会动态地推断出每层的大小
- 在编写代码时无需知道维度是什么就可以设置参数——可以大大简化定义和修改模型的任务
- **但是，pytorch中没有延后初始化**
- 延后初始化**不是深度学习中的常见做法**